In [1]:
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
from transformers import LlamaForCausalLM

#hf_hub_path = '/localdisk/ssrivas9/prismatic-vlms/runs/lora-stage-0-after-llava-test/'
hf_token = None
# llm_config = AutoConfig.from_pretrained("lmsys/vicuna-7b-v1.5", token=hf_token)
# llm_model = AutoModelForCausalLM.from_pretrained(
#         "lmsys/vicuna-7b-v1.5",
#         config=llm_config,  # Pass the modified configuration
#         token=hf_token,
#         load_in_4bit=True 
#     )
llm_config = AutoConfig.from_pretrained("lmsys/vicuna-7b-v1.5", token=hf_token, 
                 #quantization_config=self.bnb_config if self.mitigation=='qlora' else None, #
                    #torch_dtype = torch.bfloat16 if self.mitigation=='qlora' else None
            )
llm_model = LlamaForCausalLM.from_pretrained(
        "lmsys/vicuna-7b-v1.5",
        config=llm_config, 
        token=hf_token,
        load_in_4bit=True 
    )

/localdisk/ssrivas9/miniconda3/envs/eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/localdisk/ssrivas9/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/localdisk/ssrivas9/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.70s/it]
/localdisk/ssrivas9/miniconda3/envs/eval/lib/python3.10/site-packages/transformers/generat

In [2]:

#llm_model = LlamaForCausalLM(config=llm_config) #LlamaForCausalLM._from_config(llm_config)

tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5", model_max_length=2048, token=hf_token)
#tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5")

tokenizer.add_special_tokens({"pad_token": "<PAD>"})
llm_model.config.pad_token_id = tokenizer.pad_token_id
llm_model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=64)
# Increae the 
from peft import (
    LoraConfig,
    PrefixTuningConfig, #Prefix-Tuning
    PromptEncoderConfig, #P-Tuning
    PromptTuningConfig, # Prompt Tuning
    IA3Config,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
lora_r=16
lora_target_modules= 'all-linear'#["q_proj", "v_proj"]
lora_alpha=8
lora_dropout=0.05
# Add PEFT LoRA on top of this.
llm_model = prepare_model_for_kbit_training(llm_model)
loraconfig = LoraConfig(
    r=lora_r, lora_alpha=lora_alpha, target_modules=lora_target_modules,
    lora_dropout=lora_dropout, bias="none", task_type="CAUSAL_LM"
)
llm_model = get_peft_model(llm_model, loraconfig)

In [3]:
import torch
model_path = "/localdisk/ssrivas9/prismatic-vlms/runs/qlora-stage-0-after-llava-test/checkpoints/latest-checkpoint.pt"
#model_path = "/localdisk/ssrivas9/prismatic-vlms/runs/lora-stage-0-after-llava-vqav2/checkpoints/latest-checkpoint.pt"
model_state_dict = torch.load(model_path, map_location="cpu")

In [4]:
import torch

#model_path = "/localdisk/ssrivas9/prismatic-vlms/runs/qlora-stage-0-after-llava-test/checkpoints/latest-checkpoint.pt"

model_state_dicts = model_state_dict['model']
model_state_dicts.keys()

dict_keys(['projector', 'llm_backbone'])

In [5]:
model_state_dicts['llm_backbone'].keys()

odict_keys(['llm.base_model.model.model.embed_tokens.weight', 'llm.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'llm.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.absmax', 'llm.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.quant_map', 'llm.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.quant_state.bitsandbytes__fp4', 'llm.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'llm.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'llm.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'llm.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.absmax', 'llm.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.quant_map', 'llm.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight.quant_state.bitsandbytes__fp4', 'llm.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'llm.base_model.mo

In [6]:
new_model_state_dict = {}
for k, v in model_state_dicts['llm_backbone'].items():
    if k.startswith('llm.'):
        new_model_state_dict[k.replace('llm.', '')] = v
    else:
        new_model_state_dict[k] = v

In [7]:
llm_model.load_state_dict(new_model_state_dict)

<All keys matched successfully>

In [20]:
for key, param in model_state_dict['model'].items():
    for mkey in model_state_dicts:
        if key.startswith(mprefix := f"{mkey}."):
            model_state_dicts[mkey][key.removeprefix(mprefix)] = param

In [21]:
# Replace the llm.base_model... with just base_model..
new_model_state_dict = {}
for k, v in model_state_dict['model']['llm_backbone'].items():
    new_model_state_dict[k.replace('llm.','')] = v

In [22]:
llm_model.load_state_dict(new_model_state_dict)

<All keys matched successfully>

: 

In [ ]:
llm_model